In [22]:
from Pylette import extract_colors

colors = extract_colors("train_data/jungle/2385_2339_2887.png", palette_size=5, mode="MedianCut")
print(colors)

(9, 34, 2, 0.5) 
(18, 71, 4, 0.25) 
(30, 78, 10, 0.125) 
(49, 58, 25, 0.0625) 
(71, 72, 37, 0.0625) 

